<h1 align = "center">Intuitive Understanding of Randomized Singular Value Decomposition</h1>

<h4 align = "center">A Python Implementation of SVD with Randomized Linear Algebra</h4>

Matrix decomposition is a foundational tool for some critical applications like data compression, dimensionality reduction, and sparsity learning. In many cases, for purposes of approximating a data matrix by a low-rank structure, the Singular Value Decomposition (SVD) is the best choice. However, the accurate and efficient SVD of large-scale datasets is computationally challenging. To resolve the SVD in this situation, there are many algorithms have been developed by applying randomized linear algebra. One of the most important algorithms is randomized SVD, which is competitively efficient for factorizing any matrix with a relatively low rank.

<p align="center">
<img align="middle" src="https://github.com/xinychen/tensor-learning/blob/master/images/svd_history.png" width="700" />
</p>

<h6 align="center">
<b>Figure 1</b>: A timeline of major SVD developments. (The picture is from [2]).
</h6>


This post will introduce the preliminary and essential idea of the randomized SVD. To help readers gain a better understanding of randomized SVD, we also provide the corresponding Python implementation in this post.

> For reproducing this notebook, please clone or download the **tensor-learning** repository ([https://github.com/xinychen/tensor-learning](https://github.com/xinychen/tensor-learning)) on your computer first.

## Preliminary

### SVD Formula

As you may already know, SVD is one of the most important decomposition formula in linear algebra. For any given matrix $\boldsymbol{A}$, SVD has the form of
\begin{equation}
\boldsymbol{A}=\boldsymbol{U}\boldsymbol{\Sigma}\boldsymbol{V}^\top
\end{equation}
where the matrices $\boldsymbol{U}$ and $\boldsymbol{V}$ consists of left and right singular vectors, respectively. The diagonal entries of $\boldsymbol{\Sigma}$ are singular values.

### A Small Matrix Example

Take a 3-by-3 matrix for example, we can compute the SVD by using `numpy.linalg.svd` in Python. Let us have a look:

In [1]:
import numpy as np
A = np.array([[1, 3, 2],
              [5, 3, 1],
              [3, 4, 5]])
u, s, v = np.linalg.svd(A, full_matrices = 0)
print('Left singular vectors:')
print(u)
print()
print('Singular values:')
print(s)
print()
print('Right singular vectors:')
print(v)
print()

Left singular vectors:
[[-0.37421754  0.28475648 -0.88253894]
 [-0.56470638 -0.82485997 -0.02669705]
 [-0.7355732   0.48838486  0.46948087]]

Singular values:
[9.34265841 3.24497827 1.08850813]

Right singular vectors:
[[-0.57847229 -0.61642675 -0.53421706]
 [-0.73171177  0.10269066  0.67383419]
 [ 0.36051032 -0.78068732  0.51045041]]



In this case, the singular values are 9.3427, 3.2450, and 1.0885.

## Randomized SVD

### Essential Idea

Randomized SVD can be broken into three steps. For any given $m$-by-$n$ matrix $\boldsymbol{A}$, if we impose a target rank $k$ with $k < \min\{m, n\}$, then the first step is to

- 1) generate a Gaussian random matrix $\boldsymbol{Ω}$ with size of $n$-by-$k$,
- 2) compute a new $m$-by-$k$ matrix $\boldsymbol{Y}$,
- and 3) apply QR decomposition to the matrix $\boldsymbol{Y}$.

Note that the first step needs to return the $m$-by-$k$ matrix $\boldsymbol{Q}$.

<p align="center">
<img align="middle" src="https://github.com/xinychen/tensor-learning/blob/master/images/rsvd_step1.png" width="600" />
</p>

<h6 align="center">
<b>Figure 2</b>: The first step of randomized SVD. (The picture is from [2]).
</h6>


In [2]:
import numpy as np
def rsvd(A, Omega):
    Y = A @ Omega
    Q, _ = np.linalg.qr(Y)
    B = Q.T @ A
    u_tilde, s, v = np.linalg.svd(B, full_matrices = 0)
    u = Q @ u_tilde
    return u, s, v

In [3]:
np.random.seed(1000)
A = np.array([[1, 3, 2],
              [5, 3, 1],
              [3, 4, 5]])
rank = 2
Omega = np.random.randn(A.shape[1], rank)
u, s, v = rsvd(A, Omega)
print('Left singular vectors:')
print(u)
print()
print('Singular values:')
print(s)
print()
print('Right singular vectors:')
print(v)
print()

Left singular vectors:
[[ 0.38070859  0.60505354]
 [ 0.56830191 -0.74963644]
 [ 0.72944767  0.26824507]]

Singular values:
[9.34224023 3.02039888]

Right singular vectors:
[[ 0.57915029  0.61707064  0.53273704]
 [-0.77420021  0.21163814  0.59650929]]



In [4]:
import numpy as np

def power_iteration(A, Omega, power_iter = 3):
    Y = A @ Omega
    for q in range(power_iter):
        Y = A @ (A.T @ Y)
    Q, _ = np.linalg.qr(Y)
    return Q

def rsvd(A, Omega):
    Q = power_iteration(A, Omega)
    B = Q.T @ A
    u_tilde, s, v = np.linalg.svd(B, full_matrices = 0)
    u = Q @ u_tilde
    return u, s, v

In [5]:
np.random.seed(1000)

A = np.array([[1, 3, 2],
              [5, 3, 1],
              [3, 4, 5]])
rank = 2
Omega = np.random.randn(A.shape[1], rank)
u, s, v = rsvd(A, Omega)
print('Left singular vectors:')
print(u)
print()
print('Singular values:')
print(s)
print()
print('Right singular vectors:')
print(v)
print()

Left singular vectors:
[[ 0.37421757  0.28528579]
 [ 0.56470638 -0.82484381]
 [ 0.73557319  0.48810317]]

Singular values:
[9.34265841 3.24497775]

Right singular vectors:
[[ 0.57847229  0.61642675  0.53421706]
 [-0.73178429  0.10284774  0.67373147]]



### 2. The central idea behind Randomized SVD


#### Power Iterations



In [1]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

def power_iteration(mat, Phi, power_iter = 3):
    B = mat @ Phi
    for q in range(power_iter):
        B = mat @ (mat.T @ B)
    Q, _ = np.linalg.qr(B)
    return Q

### Randomized Singular Value Decomposition

In [7]:
def rsvd(mat, rank, power_iter):
    dim1, dim2 = mat.shape
    Phi = np.random.randn(dim2, rank)
    A = mat @ Phi
    if power_iter > 0:
        for k in range(power_iter):
            A = mat @ (mat.T @ A)
    Q, R = np.linalg.qr(A)
    u_tilde, s, v = np.linalg.svd(Q.T @ mat, full_matrices = 0)
    return Q @ u_tilde, s, v

### Small Worked Example

We will give a sufficiently detailed understanding with a small worked example. The problem is a simple SVD of 5-by-4 matrix, i.e.,
$$\boldsymbol{X}=\left(\begin{array}{cccc}
1 & 3 & 2 & 4 \\
5 & 3 & 1 & 2 \\
3 & 4 & 5 & 2 \\
4 & 4 & 2 & 1 \\
4 & 2 & 3 & 3 \\
\end{array}\right)\in\mathbb{R}^{5\times 4}.$$

In [9]:
mat = np.array([[1, 3, 2, 4],
                [5, 3, 1, 2],
                [3, 4, 5, 2],
                [4, 4, 2, 1],
                [4, 2, 3, 3]])
u, s, v = np.linalg.svd(mat, full_matrices = 0)
print('Left singular vectors:')
print(u)
print()
print('Singular values:')
print(s)
print()
print('Right singular vectors:')
print(v)
print()

Left singular vectors:
[[-0.35579275  0.61467653  0.58103306 -0.39692223]
 [-0.43905533 -0.5883267   0.34849729 -0.03803043]
 [-0.53040969  0.37421029 -0.65683124  0.0736076 ]
 [-0.44100936 -0.36870485 -0.24920217 -0.50936999]
 [-0.45256849  0.00823753  0.21776416  0.75903265]]

Singular values:
[13.1975984   3.6191375   2.70009861  1.85329644]

Right singular vectors:
[[-0.58469804 -0.5436866  -0.4578419  -0.39104205]
 [-0.7311674   0.0324791   0.49718495  0.46598977]
 [ 0.0841724  -0.14814801 -0.59949834  0.78202872]
 [ 0.34122931 -0.82547087  0.42870697  0.1355387 ]]



In [10]:
rank = 3
power_iter = 3
u, s, v = rsvd(mat, rank, power_iter)
print('Left singular vectors:')
print(u)
print()
print('Singular values:')
print(s)
print()
print('Right singular vectors:')
print(v)
print()

Left singular vectors:
[[-0.35579222 -0.61522273  0.58260656]
 [-0.43905528  0.58827643  0.34864337]
 [-0.53040979 -0.37411357 -0.65711692]
 [-0.44100867  0.36799365 -0.24717929]
 [-0.45256952 -0.00717968  0.21474902]]

Singular values:
[13.1975984   3.61913492  2.70008736]

Right singular vectors:
[[-0.5846981  -0.54368644 -0.45784198 -0.39104207]
 [ 0.73141086 -0.03306811 -0.49688356 -0.46588774]
 [ 0.08323863 -0.14589787 -0.60066191  0.78165876]]



In [11]:
mat = np.array([[1, 3, 2],
                [5, 3, 1],
                [3, 4, 5]])
u, s, v = np.linalg.svd(mat, full_matrices = 0)
print('Left singular vectors:')
print(u)
print()
print('Singular values:')
print(s)
print()
print('Right singular vectors:')
print(v)
print()

Left singular vectors:
[[-0.37421754  0.28475648 -0.88253894]
 [-0.56470638 -0.82485997 -0.02669705]
 [-0.7355732   0.48838486  0.46948087]]

Singular values:
[9.34265841 3.24497827 1.08850813]

Right singular vectors:
[[-0.57847229 -0.61642675 -0.53421706]
 [-0.73171177  0.10269066  0.67383419]
 [ 0.36051032 -0.78068732  0.51045041]]



In [12]:
rank = 2
power_iter = 3
u, s, v = rsvd(mat, rank, power_iter)
print('Left singular vectors:')
print(u)
print()
print('Singular values:')
print(s)
print()
print('Right singular vectors:')
print(v)
print()

Left singular vectors:
[[ 0.37421764  0.28389005]
 [ 0.56470638 -0.82488578]
 [ 0.73557315  0.48884546]]

Singular values:
[9.34265841 3.24497688]

Right singular vectors:
[[ 0.57847229  0.61642676  0.53421705]
 [-0.73159303  0.1024336   0.67400222]]



In [34]:
import time

mat = np.random.rand(10000, 9000)
start = time.time()
U, S, V = fast_svd(mat)
end = time.time()
print(np.diag(S[:10]))
print(end - start)

[4743.47345882   56.30120971   56.14880174   56.13332907   56.03742922
   56.00312683   55.98918016   55.8925917    55.86384718   55.80551466]
387.3423180580139


In [43]:
import time

start = time.time()
U, S, V = rsvd(mat, 100, 2)
end = time.time()
print(np.diag(S[:10]))
print(end - start)

[4743.47345882   52.08820533   52.03989705   51.98184477   51.84658782
   51.78052861   51.68910186   51.59906955   51.51632298   51.43246594]
1.8465938568115234


### Summary

In this post, you discovered the randomized linear algebra method for SVD.

Specifically, you learned:

- The essential idea of randomized SVD.
- How to implement randomized SVD step-by-step.

Do you have any question? Ask your question by creating an issue at the **tensor-learning** repository ([https://github.com/xinychen/tensor-learning](https://github.com/xinychen/tensor-learning)) and I will do my best to answer. If you find these codes useful, please star (★) this repository.

### References

[1] Per-Gunnar Martinsson (2016). Randomized methods for matrix computations and analysis of high dimensional data. arXiv:1607.01649. [[PDF](https://arxiv.org/pdf/1607.01649v1.pdf)]

[2] N. Benjamin Erichson, Sergey Voronin, Steven L. Brunton, J. Nathan Kutz (2016). Randomized Matrix Decompositions Using R. arXiv:1608.02148. [[PDF](https://arxiv.org/pdf/1608.02148.pdf)]